<a href="https://colab.research.google.com/github/Meridor6919/BeeClassification/blob/master/doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preview of CNN based on bee_classification
-------------------

## Spis treści
1. [Wstęp](#1)
    1. [Cel projektu](#1.1)
    2. [Czym jest CNN?](#1.2)
    3. [Z czego składa się bee_dataset?](#1.3)
2. [Model](#2)
    1. [Specyfika modelu](#2.1)
    2. [Część eksperymentalna](#2.2)
      1. [Architektura](#2.2.1)
      2. [Porównanie jakości klasyfikacji](#2.2.2)
    3. [Użytkowanie](#2.3)
      1. [Przygotowanie danych](#2.3.1)
      2. [Opis input-output](#2.3.2)
3. [Contributing](#3)
4. [Źródła](#4)

<a name="1"></a>
# Wstęp
-------------------

<a name="1.1"></a>
## Cel projektu
-------------------

krótki opis

<a name="1.2"></a>
## Czym jest CNN?
-------------------

geron 14

<a name="1.3"></a>
## Z czego składa się bee_dataset?
-------------------

[bee_dataset](https://www.tensorflow.org/datasets/catalog/bee_dataset)

<a name="2"></a>
# Model
-------------------

<a name="2.1"></a>
## Specyfika modelu
-------------------

<a name="2.2"></a>
## Część eksperymentalna
-------------------

<a name="2.2.1"></a>
### Architektura
-------------------

<a name="2.2.2"></a>
### Porównanie jakości klasyfikacji
-------------------

<a name="2.3"></a>
## Użytkowanie
-------------------

<a name="2.3.1"></a>
### Przygotowanie danych
-------------------

<a name="2.3.2"></a>
### Opis input-output
-------------------

<a name="3"></a>
# Contributing
-------------------

<a name="4"></a>
# Źródła
-------------------
1. [Deep Learning with Python by François Chollet, version 6, 2017](https://www.manning.com/books/deep-learning-with-python)

2. [Hands-On Machine Learning with Scikit-Learn, Keras and TensorFlow by Aurélien Géron, 2nd edition, 2019](https://www.oreilly.com/library/view/hands-on-machine-learning/9781492032632/)

3. [TensorFlow Keras Documentation](https://www.tensorflow.org/api_docs/python/tf/keras)


<br/>

![zdjecie](https://drive.google.com/uc?id=16xYDkCsPLDC8VhVVwB5bq3rpxdC-fnYM)
// "https://drive.google.com/uc?id=" format odnośnika